# Data Preprocessing and Cleaning for BI Dashboard

## Setup

In [ ]:
# ============================================
# SETUP & IMPORTS
# ============================================

import pandas as pd
import numpy as np
import os
import re

# Define file paths based on your structure
RAW_DATA_DIR = '../data'
PROCESSED_DATA_DIR = '../data/processed'

# Create processed directory if it doesn't exist
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

print(f"Working Directory: {os.getcwd()}")
print(f"Data Source: {RAW_DATA_DIR}")
print(f"Export Target: {PROCESSED_DATA_DIR}")
print("\n✓ Setup complete")

## Phase 1: Data Ingestion

In [ ]:
# ============================================
# PHASE 1: DATA INGESTION
# ============================================

# Helper function for flexible sheet detection
def find_sheet(xls_file, possible_names):
    """Find sheet by trying multiple name variations"""
    available = xls_file.sheet_names
    for name in possible_names:
        # Exact match
        if name in available:
            return name
        # Case-insensitive match
        for sheet in available:
            if sheet.lower() == name.lower():
                return sheet
    # If not found, return first sheet as fallback
    print(f"   ⚠️  None of {possible_names} found. Using first sheet: {available[0]}")
    return available[0]

# --- Load Student Survey ---
student_path = os.path.join(RAW_DATA_DIR, 'student_survey.xlsx')
student_xls = pd.ExcelFile(student_path)

print(f"📂 Student Survey File: {student_path}")
print(f"   Available sheets: {student_xls.sheet_names}")

# Find the data and codebook sheets
data_sheet = find_sheet(student_xls, ['Data', 'data', 'DATA', 'responses', 'raw_data'])
codebook_sheet = find_sheet(student_xls, ['Codebook', 'codebook', 'CODEBOOK', 'metadata', 'dictionary'])

print(f"   ✓ Using data sheet: '{data_sheet}'")
print(f"   ✓ Using codebook sheet: '{codebook_sheet}'")

# Read the sheets
df_student_raw = pd.read_excel(student_xls, sheet_name=data_sheet)
df_student_meta = pd.read_excel(student_xls, sheet_name=codebook_sheet)

print(f"   Student data shape: {df_student_raw.shape}")
print(f"   Student codebook shape: {df_student_meta.shape}")

# --- Load Teacher Burnout ---
teacher_path = os.path.join(RAW_DATA_DIR, 'teacher_burnout.xlsx')
teacher_xls = pd.ExcelFile(teacher_path)

print(f"\n📂 Teacher Burnout File: {teacher_path}")
print(f"   Available sheets: {teacher_xls.sheet_names}")

# Find the data and subtitles sheets
data_sheet_t = find_sheet(teacher_xls, ['Data base', 'Data_base', 'Database', 'data', 'Data'])
subtitle_sheet = find_sheet(teacher_xls, ['Subtitles (codes)', 'Subtitles', 'codes', 'Codes', 'legend'])

print(f"   ✓ Using data sheet: '{data_sheet_t}'")
print(f"   ✓ Using subtitle sheet: '{subtitle_sheet}'")

# Read the sheets
df_teacher_raw = pd.read_excel(teacher_xls, sheet_name=data_sheet_t)
df_teacher_meta = pd.read_excel(teacher_xls, sheet_name=subtitle_sheet)

print(f"   Teacher data shape: {df_teacher_raw.shape}")
print(f"   Teacher subtitles shape: {df_teacher_meta.shape}")

print("\n" + "="*80)
print("✅ PHASE 1 COMPLETE - Data Ingestion Successful")
print("="*80)

## Phase 1.5 DATA STRUCTURE INSPECTION

In [ ]:
# ============================================
# PHASE 1.5: DATA STRUCTURE INSPECTION
# ============================================

print("="*80)
print("STUDENT SURVEY - CODEBOOK STRUCTURE")
print("="*80)
print(f"Columns: {df_student_meta.columns.tolist()}")
print(f"Shape: {df_student_meta.shape}")
display(df_student_meta.head(15))

print("\n" + "="*80)
print("STUDENT SURVEY - RAW DATA STRUCTURE")
print("="*80)
print(f"Columns: {df_student_raw.columns.tolist()[:20]}...")  # First 20 columns
print(f"Shape: {df_student_raw.shape}")
display(df_student_raw.head(3))

print("\n" + "="*80)
print("TEACHER BURNOUT - SUBTITLES (CODES) STRUCTURE")
print("="*80)
print(f"Columns: {df_teacher_meta.columns.tolist()}")
print(f"Shape: {df_teacher_meta.shape}")
display(df_teacher_meta.head(3))

print("\n" + "="*80)
print("TEACHER BURNOUT - RAW DATA STRUCTURE")
print("="*80)
print(f"Columns: {df_teacher_raw.columns.tolist()}")
print(f"Shape: {df_teacher_raw.shape}")
display(df_teacher_raw.head(3))

## PHASE 2: MAPPING LOGIC

In [ ]:
# ============================================
# PHASE 2: MAPPING LOGIC
# ============================================

def parse_student_values(values_str):
    """
    Parses student codebook 'values' column.
    Example: '1: "Full-time", 2: "Part-time"' → {1: 'Full-time', 2: 'Part-time'}
    """
    if pd.isna(values_str) or values_str.strip() == '':
        return {}
    
    mapping = {}
    
    # Split by comma (handles multiple value pairs)
    pairs = values_str.split(',')
    
    for pair in pairs:
        if ':' in pair:
            try:
                # Split by first colon only
                code_part, label_part = pair.split(':', 1)
                
                # Clean up
                code = code_part.strip()
                label = label_part.strip().strip('"').strip("'")
                
                # Convert code to integer if possible
                try:
                    code = int(code)
                except ValueError:
                    pass
                
                mapping[code] = label
            except Exception as e:
                print(f"Warning: Could not parse pair '{pair}': {e}")
                continue
    
    return mapping


def create_student_mappings(codebook_df):
    """
    Creates {column_name: {code: label}} dictionary from student codebook.
    """
    mappings = {}
    
    for _, row in codebook_df.iterrows():
        col_name = row['name']
        values_str = row['values']
        
        # Parse the values string
        value_map = parse_student_values(values_str)
        
        if value_map:  # Only add if we got valid mappings
            mappings[col_name] = value_map
    
    return mappings


def parse_teacher_subtitle(subtitle_text):
    """
    Parses teacher subtitle format.
    Example: "Female (1) Male (2)" → {1: 'Female', 2: 'Male'}
    Example: "Single (1) Married or married (2) Divorced or separated (3) Widower (4)"
    """
    if pd.isna(subtitle_text) or subtitle_text.strip() == '':
        return {}
    
    mapping = {}
    
    # Find all patterns like "Label (number)"
    # This regex captures: word/phrase followed by (number)
    pattern = r'([A-Za-z\s\-/]+)\s*\((\d+)\)'
    matches = re.findall(pattern, subtitle_text)
    
    for label, code in matches:
        label = label.strip()
        code = int(code)
        mapping[code] = label
    
    return mapping


def create_teacher_mappings(subtitle_df):
    """
    Creates {column_name: {code: label}} dictionary from teacher subtitles.
    Note: The subtitle sheet has ONE ROW where each column contains the coding info.
    """
    mappings = {}
    
    # Get the first row (index 0) which contains all the subtitle information
    subtitle_row = subtitle_df.iloc[0]
    
    for col_name in subtitle_df.columns:
        subtitle_text = str(subtitle_row[col_name])
        
        # Parse the subtitle
        value_map = parse_teacher_subtitle(subtitle_text)
        
        if value_map:  # Only add if we got valid mappings
            mappings[col_name] = value_map
    
    return mappings


print("✓ Custom parsing functions defined")
print("\nTesting student parser:")
test_student = '1: "Full-time", 2: "Part-time"'
print(f"  Input: {test_student}")
print(f"  Output: {parse_student_values(test_student)}")

print("\nTesting teacher parser:")
test_teacher = "Female (1) Male (2)"
print(f"  Input: {test_teacher}")
print(f"  Output: {parse_teacher_subtitle(test_teacher)}")

## PHASE 3A: DECODE STUDENT DATA

In [ ]:
# ============================================
# PHASE 3A: DECODE STUDENT DATA
# ============================================

# Create mappings from the codebook
student_mappings = create_student_mappings(df_student_meta)

print(f"Created mappings for {len(student_mappings)} student columns")
print(f"Sample columns with mappings: {list(student_mappings.keys())[:5]}")

# Create a copy to work on
df_student_clean = df_student_raw.copy()

# Apply mappings
decoded_count = 0
for col in df_student_clean.columns:
    if col in student_mappings:
        print(f"  Decoding: {col}")
        # Map values, keep original if no match found
        df_student_clean[col] = df_student_clean[col].map(student_mappings[col]).fillna(df_student_clean[col])
        decoded_count += 1

print(f"\n✓ Decoded {decoded_count} columns in student data")

# Add metadata columns
df_student_clean['User_Type'] = 'Student'
df_student_clean['Dataset_Source'] = 'Global_Student_Survey_COVID19'

print("\n--- STUDENT DATA SAMPLE (DECODED) ---")
# Show some key columns if they exist
sample_cols = [col for col in ['Q3', 'Q4', 'Q5', 'Q8', 'User_Type'] if col in df_student_clean.columns]
if sample_cols:
    display(df_student_clean[sample_cols].head(5))
else:
    display(df_student_clean.iloc[:, :5].head(5))

print(f"\nFinal Student Data Shape: {df_student_clean.shape}")

## Phase 3B - Decode Teacher Data

In [ ]:
# ============================================
# PHASE 3B: DECODE TEACHER DATA
# ============================================

# Create mappings from the subtitles
teacher_mappings = create_teacher_mappings(df_teacher_meta)

print(f"Created mappings for {len(teacher_mappings)} teacher columns")
print(f"Columns with mappings: {list(teacher_mappings.keys())}")

# Show one example mapping
if teacher_mappings:
    first_key = list(teacher_mappings.keys())[0]
    print(f"\nExample - {first_key} mapping: {teacher_mappings[first_key]}")

# Create a copy to work on
df_teacher_clean = df_teacher_raw.copy()

# Apply mappings
decoded_count = 0
for col in df_teacher_clean.columns:
    if col in teacher_mappings:
        print(f"  Decoding: {col}")
        df_teacher_clean[col] = df_teacher_clean[col].map(teacher_mappings[col]).fillna(df_teacher_clean[col])
        decoded_count += 1

print(f"\n✓ Decoded {decoded_count} columns in teacher data")

# Add metadata columns
df_teacher_clean['User_Type'] = 'Teacher'
df_teacher_clean['Dataset_Source'] = 'Brazilian_Teacher_Burnout_COVID19'

print("\n--- TEACHER DATA SAMPLE (DECODED) ---")
# Show first few columns that likely got decoded
display(df_teacher_clean.iloc[:, :8].head(5))

print(f"\nFinal Teacher Data Shape: {df_teacher_clean.shape}")

## PHASE 3C: INSPECT COLUMNS

In [ ]:
# ============================================
# PHASE 3C: INSPECT COLUMNS FOR STANDARDIZATION
# ============================================

print("STUDENT COLUMNS:")
print("="*80)
for i, col in enumerate(df_student_clean.columns, 1):
    print(f"{i:3d}. {col}")

print("\n\nTEACHER COLUMNS:")
print("="*80)
for i, col in enumerate(df_teacher_clean.columns, 1):
    print(f"{i:3d}. {col}")

print("\n\n" + "="*80)
print("ACTION REQUIRED:")
print("="*80)
print("Review the columns above and identify:")
print("1. Which student columns correspond to which teacher columns?")
print("2. Which columns contain demographic info (age, gender, etc.)?")
print("3. Which columns contain outcomes (burnout scores, satisfaction, etc.)?")
print("\nWe'll use this information in the next cell to create standardized names.")

## PHASE 4: DATA INTEGRATION & STANDARDIZATION

In [ ]:
# ============================================
# PHASE 4: DATA INTEGRATION & STANDARDIZATION
# ============================================

# ========== STEP 1: RENAME COLUMNS ==========
# Based on the actual columns, create meaningful standardized names
# IMPORTANT: Update these mappings based on what you saw in Cell 5.5

# Student Survey - Based on Q-codes, map to meaningful names
student_renames = {
    'Q1': 'Country_Study',
    'Q3': 'Is_Citizen',
    'Q4': 'Student_Status',  # Full-time/Part-time
    'Q5': 'Study_Level',     # Bachelor/Master/Doctoral
    'Q6': 'Field_of_Study',
    'Q7': 'Age',
    'Q8': 'Gender',
    'Q9': 'Had_Onsite_Classes',
    # Add more Q-codes as you identify them from Cell 5.5
    # Example: 'Q10a': 'Satisfaction_RealTime_Video', etc.
}

# Teacher Burnout - Based on visible columns
teacher_renames = {
    'Age:': 'Age',
    'Sex:': 'Gender',
    'Marital status:': 'Marital_Status',
    'I live with:': 'Living_Situation',
    'Area of knowledge in which it operates:': 'Knowledge_Area',
    'Level of training (titration)': 'Education_Level',
    'Main level of education in which it operates:': 'Teaching_Level',
    # Add burnout-related columns here as you identify them
}

# Apply renames (only rename columns that exist)
student_renames_filtered = {k: v for k, v in student_renames.items() if k in df_student_clean.columns}
teacher_renames_filtered = {k: v for k, v in teacher_renames.items() if k in df_teacher_clean.columns}

df_student_standardized = df_student_clean.rename(columns=student_renames_filtered)
df_teacher_standardized = df_teacher_clean.rename(columns=teacher_renames_filtered)

print(f"✓ Column names standardized")
print(f"  Student: {len(student_renames_filtered)} columns renamed")
print(f"  Teacher: {len(teacher_renames_filtered)} columns renamed")


# ========== STEP 2: CREATE AGE GROUPS ==========
def create_age_groups(age_value):
    """Converts age to standard groups for comparison"""
    try:
        age = int(float(age_value))
        if age < 25:
            return '18-24'
        elif age < 35:
            return '25-34'
        elif age < 45:
            return '35-44'
        elif age < 55:
            return '45-54'
        else:
            return '55+'
    except (ValueError, TypeError):
        return 'Unknown'

# Apply age grouping if Age column exists
if 'Age' in df_student_standardized.columns:
    df_student_standardized['Age_Group'] = df_student_standardized['Age'].apply(create_age_groups)
    df_student_standardized['Age_Numeric'] = pd.to_numeric(df_student_standardized['Age'], errors='coerce')
    print("✓ Student age groups created")

if 'Age' in df_teacher_standardized.columns:
    df_teacher_standardized['Age_Group'] = df_teacher_standardized['Age'].apply(create_age_groups)
    df_teacher_standardized['Age_Numeric'] = pd.to_numeric(df_teacher_standardized['Age'], errors='coerce')
    print("✓ Teacher age groups created")


# ========== STEP 3: STANDARDIZE GENDER ==========
def standardize_gender(gender_value):
    """Standardizes gender values"""
    if pd.isna(gender_value):
        return 'Unknown'
    
    gender_str = str(gender_value).lower().strip()
    
    if 'male' in gender_str and 'female' not in gender_str:
        return 'Male'
    elif 'female' in gender_str:
        return 'Female'
    elif 'diverse' in gender_str or 'other' in gender_str:
        return 'Gender Diverse'
    elif 'prefer not' in gender_str:
        return 'Prefer Not to Say'
    else:
        return 'Unknown'

if 'Gender' in df_student_standardized.columns:
    df_student_standardized['Gender_Standardized'] = df_student_standardized['Gender'].apply(standardize_gender)
    print("✓ Student gender standardized")

if 'Gender' in df_teacher_standardized.columns:
    df_teacher_standardized['Gender_Standardized'] = df_teacher_standardized['Gender'].apply(standardize_gender)
    print("✓ Teacher gender standardized")


# ========== STEP 4: CREATE FINAL DATASETS ==========
df_student_final = df_student_standardized.copy()
df_teacher_final = df_teacher_standardized.copy()

print(f"\n✓ Standardization complete")
print(f"  Student final shape: {df_student_final.shape}")
print(f"  Teacher final shape: {df_teacher_final.shape}")

## PHASE 4.5: DATA QUALITY VALIDATION

In [ ]:
# ============================================
# PHASE 4.5: DATA QUALITY VALIDATION
# ============================================

def quality_report(df, dataset_name):
    """Comprehensive data quality report"""
    print(f"\n{'='*80}")
    print(f"DATA QUALITY REPORT: {dataset_name}")
    print(f"{'='*80}")
    
    print(f"\n📊 BASIC INFO:")
    print(f"  Total Records: {len(df):,}")
    print(f"  Total Columns: {len(df.columns)}")
    print(f"  Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print(f"\n❌ MISSING VALUES:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing_Count': missing,
        'Missing_Percent': missing_pct
    })
    missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
    
    if len(missing_df) > 0:
        print(missing_df.head(10))
    else:
        print("  ✓ No missing values!")
    
    print(f"\n🔁 DUPLICATES:")
    dup_count = df.duplicated().sum()
    print(f"  Duplicate rows: {dup_count}")
    
    print(f"\n📈 DATA TYPES:")
    print(df.dtypes.value_counts())
    
    print(f"\n🎯 KEY COLUMNS CHECK:")
    key_cols = ['Age', 'Gender', 'User_Type', 'Age_Group', 'Gender_Standardized']
    for col in key_cols:
        if col in df.columns:
            print(f"  {col}: {df[col].nunique()} unique values")
            value_counts = df[col].value_counts().head(3).to_dict()
            print(f"    Sample: {value_counts}")

# Run reports
quality_report(df_student_final, "STUDENTS (Final)")
quality_report(df_teacher_final, "TEACHERS (Final)")

## PHASE 5: EXPORT TO CSV

In [ ]:
# ============================================
# PHASE 5: EXPORT TO CSV
# ============================================

# Define output paths
student_out_path = os.path.join(PROCESSED_DATA_DIR, 'students_cleaned.csv')
teacher_out_path = os.path.join(PROCESSED_DATA_DIR, 'teachers_cleaned.csv')

# Export individual datasets
df_student_final.to_csv(student_out_path, index=False, encoding='utf-8-sig')
df_teacher_final.to_csv(teacher_out_path, index=False, encoding='utf-8-sig')

print("✓ Successfully exported individual datasets:")
print(f"  1. {student_out_path}")
print(f"  2. {teacher_out_path}")


# ========== CREATE UNIFIED DATASET ==========
# For BI tools, create a harmonized schema with common dimensions

# Define core dimensions that exist in both (or can be added)
core_dimensions = [
    'User_Type',
    'Dataset_Source',
    'Age',
    'Age_Group',
    'Age_Numeric',
    'Gender',
    'Gender_Standardized',
]

# Add these columns to both if missing
for col in core_dimensions:
    if col not in df_student_final.columns:
        df_student_final[col] = None
    if col not in df_teacher_final.columns:
        df_teacher_final[col] = None

# Select only core columns for combined dataset
df_combined = pd.concat([
    df_student_final[core_dimensions],
    df_teacher_final[core_dimensions]
], ignore_index=True)

combined_out_path = os.path.join(PROCESSED_DATA_DIR, 'combined_demographics.csv')
df_combined.to_csv(combined_out_path, index=False, encoding='utf-8-sig')

print(f"  3. {combined_out_path} (Common demographics only)")


# ========== CREATE DATA DICTIONARY ==========
def create_data_dictionary(df, dataset_name):
    """Creates metadata about the dataset"""
    data_dict = pd.DataFrame({
        'Column_Name': df.columns,
        'Data_Type': df.dtypes.astype(str),
        'Non_Null_Count': df.count(),
        'Null_Count': df.isnull().sum(),
        'Unique_Values': [df[col].nunique() for col in df.columns],
        'Sample_Value_1': [df[col].dropna().iloc[0] if len(df[col].dropna()) > 0 else None for col in df.columns],
        'Sample_Value_2': [df[col].dropna().iloc[1] if len(df[col].dropna()) > 1 else None for col in df.columns],
    })
    
    dict_path = os.path.join(PROCESSED_DATA_DIR, f'data_dictionary_{dataset_name}.csv')
    data_dict.to_csv(dict_path, index=False)
    return dict_path

dict_student = create_data_dictionary(df_student_final, 'students')
dict_teacher = create_data_dictionary(df_teacher_final, 'teachers')

print(f"  4. {dict_student}")
print(f"  5. {dict_teacher}")

print("\n" + "="*80)
print("✅ PHASE 1-5 COMPLETE - DATA READY FOR BI VISUALIZATION")
print("="*80)
print(f"\nProcessed files location: {PROCESSED_DATA_DIR}")
print("\nYou can now proceed to Phase 6 for analysis and visualization!")